# 传输层协议分析

传输层用于接收会话层的数据，在必要时将数据进行分割，并将这些数据交给网络层，且保证这些数据段有效到达对端。

## 传输层协议概览

### TCP传输控制协议

基于TCP的传输层其它协议有：

- XOT基于TCP之上的X.25协议

- DSI、IP NetBIOS、SMB、NetBIOS、ISO-TP SSP、MSRPC

- Van Jacobson压缩TCP协议

- ISO-DE ISO开发环境
  + NetBIOS
  
- TALI传输适配层接口协议

- ISO 传输协议
  + TP0\TP1\TP2\TP3\TP4


### UDP用户数据报协议

基于UDP的传输层其它协议有：

- RUDP可靠的用户数据报协议

- Mobile IP移动IP协议

### SCTP流控制传输协议（基于IP）

### IOT传输协议

- WHART传输层协议
- Zigbee传输层协议
- UDP-Lite轻量级用户数据报协议

## TCP传输控制协议

TCP是一种面向连接的、可靠的、基于字节流的传输层通信协议。TCP在RFC793中定义，在OSI模型的第4层工作。

TCP层是位于IP层之上，应用层之下的中间层。不同主机的应用层之间经常需要可靠的、像管道一样的连接，但是IP层不提供这样的流机制，而是提供不可靠的包交换。

### TCP头部

TCP提供的诸多功能，都反映在其复杂的头部中。

![网络数据分析](images\07\tcp.jpg)

- 源端口号和目的端口号：再加上Ip首部的源IP地址和目的IP地址可以唯一确定一个TCP连接
- 数据序号：表示在这个报文段中的第一个数据字节序号
- 确认序号：仅当ACK标志为1时有效。确认号表示期望收到的下一个字节的序号（这个下面再详细分析）
- 偏移：就是头部长度，有4位，跟IP头部一样，以4字节为单位。最大是60个字节
- 保留位：6位，必须为0
- 6个标志位：
  - URG-紧急指针有效
  - ACK-确认序号有效
  - PSH-接收方应尽快将这个报文交给应用层
  - RST-连接重置
  - SYN-同步序号用来发起一个连接
  - FIN-终止一个连接

- 窗口字段：16位，代表的是窗口的字节容量，也就是TCP的标准窗口最大为2^16 - 1 = 65535个字节（这个下面再详细分析）

- 校验和：源机器基于数据内容计算一个数值，收信息机要与源机器数值 结果完全一样，从而证明数据的有效性。检验和覆盖了整个的TCP报文段：这是一个强制性的字段，一定是由发送端计算和存储，并由接收端进行验证的。

- 紧急指针：是一个正偏移量，与序号字段中的值相加表示紧急数据最后一个字节的序号。TCP的紧急方式是发送端向另一端发送紧急数据的一种方式

- 选项与填充（必须为4字节整数倍，不够补0）：

最常见的可选字段的最长报文大小MSS（Maximum Segment Size），每个连接方通常都在一个报文段中指明这个选项。它指明本端所能接收的最大长度的报文段。

该选项如果不设置，默认为536（20+20+536=576字节的IP数据报）

### TCP端口

TCP通信都会使用源端口和目的端口，这些端口对应着计算机中运行的网络应用程序。

端口号是16位无符号整数，范围从0到65535.对于TCP，端口号0是保留的，不能使用。

在互联网编号分配机构（IANA）负责DNS根，IP寻址和其他互联网协议资源的全球协调。这包括为众所周知的互联网服务注册常用端口号。

端口号分为三个范围：

- 熟知的端口，从0到1023的端口；

  + 20：文件传输协议（FTP）数据传输
  + 21：文件传输协议（FTP）命令控制
  + 22：Secure Shell（SSH）安全登录
  + 23：Telnet远程登录服务，未加密的短信
  + 25：简单邮件传输协议（SMTP）电子邮件路由
  + 53：域名系统（DNS）服务
  + 80：万维网中使用的超文本传输​​协议（HTTP）
  + 110：邮局协议（POP3）
  + 119：网络新闻传输协议（NNTP）
  + 123：网络时间协议（NTP）
  + 143：互联网消息访问协议（IMAP）数字邮件的管理
  + 161：简单网络管理协议（SNMP）
  + 194：互联网中继聊天（IRC）
  + 443：通过TLS / SSL的HTTP安全（HTTPS）HTTP

- 已注册的端口；

注册的端口是1024到49151之间的端口.IEA维护着名和注册范围的官方列表。

- 动态或专用端口。

动态或私有端口是49152到65535的端口。此范围的一个常见用途是短暂使用的端口。

更多有关端口的信息，可以参考http://www.iana.org/assignments/port-numbers/

#### 本节实验操作

- 打开tcp_ports.pcap文件
- 统计该文件中共有几个端口，分别是什么？
- 分析一下172.16.16.128上使用的TCP端口号有什么样变化趋势？
- 找到wireshark菜单项“视图”-“解析名称”-“解析传输层地址”，尝试勾选和不勾选两种情况下，wireshark的分组列表中最后一列“Info”列中的内容变化。

![解析传输层地址](images/07/名称解析实验.png)

- 使用“追踪流”-“TCP流”功能，分析0~7号TCP流，完成下列表格：

|tcp流号|IP1|IP1用的端口|IP2|IP2用的端口|
|-|-|-|-|-|
|0||
|1||
|2||
|3||
|4||
|5||
|6||
|7||

### TCP协议工作过程


TCP协议操作可以分为三个阶段：

- 1.在进入数据传输阶段之前，必须在多步握手过程（连接建立）中正确建立连接；

- 2.传输数据；

- 3.数据传输完成后，连接终止关闭已建立的虚拟电路并释放所有分配的资源。


TCP连接由操作系统通过编程接口管理，该接口表示通信的本地端点，即Internet套接字。在TCP连接的生命周期中，本地端点经历一系列状态变化：

- Listening：（服务器）表示等待来自任何远程TCP和端口的连接请求。
- SYN-SENT:（客户端）表示在发送连接请求之后等待匹配的连接请求。
- SYN-RECEIVED:（服务器）表示在接收和发送连接请求之后等待确认连接请求确认。
- ESTABLISHED:（服务器和客户端）代表开放连接，接收的数据可以传递给用户。连接的数据传输阶段的正常状态。
- FIN-WAIT-1:（服务器和客户端）表示等待来自远程TCP的连接终止请求，或者确认先前发送的连接终止请求。
- FIN-WAIT-2:（服务器和客户端）表示等待来自远程TCP的连接终止请求。
- CLOSE-WAIT:（服务器和客户端）表示等待来自本地用户的连接终止请求。
- CLOSING:（服务器和客户端）表示等待来自远程TCP的连接终止请求确认。
- LAST-ACK:（服务器和客户端）表示等待先前发送到远程TCP的连接终止请求的确认（其包括对其连接终止请求的确认）。
- TIME-WAIT:（服务器或客户端）表示等待足够的时间传递以确保远程TCP收到其连接终止请求的确认。根据RFC 793，连接可以在TIME-WAIT中保持最多- - 四分钟，称为两个MSL（最大段寿命）。
- CLOSED:（服务器和客户端）根本不表示连接状态。

![网络数据分析](images\07\Tcp_state_diagram_fixed_new.svg)

#### 建立连接

为了建立连接，TCP使用三次握手。在客户端尝试连接服务器之前，服务器必须先绑定并侦听端口以打开连接：这称为被动打开。一旦建立被动打开，客户端可以启动主动打开。要建立连接，会发生三方（或三步）握手：

- 1.SYN：活动打开由客户端向服务器发送SYN执行。客户端将段的序列号设置为随机值A.
- 2.SYN-ACK：作为响应，服务器回复SYN-ACK。确认号码设置为比收到的序列号多一个，即A + 1，服务器为该数据包选择的序列号是另一个随机数，B。
- 3.ACK：最后，客户端将ACK发送回服务器。序列号被设置为接收的确认值，即A + 1，并且确认号被设置为比接收的序列号多一个，即B + 1。

此时，客户端和服务器都收到了连接确认。步骤1,2为一个方向建立连接参数（序列号）并确认。步骤2,3为另一个方向建立连接参数（序列号）并确认。通过这些，建立了全双工通信。


#### 撤销连接

连接终止阶段使用四次握手，连接的每一侧都独立终止。当端点希望停止其连接的一半时，它发送FIN数据包，另一端用ACK确认。

当TCP其中一方调用close()函数之后，TCP连接进入四次挥手阶段。

- FIN_WAIT_1，连接中断主动方的状态。

调用close()函数，发出连接中断请求FIN报文后进入该状态，等待对方的响应。

- FIN_WAIT_2，连接中断主动方的状态。

收到了对方的中断请求ACK报文后进入该状态，等待对方的FIN报文。

- TIME_WAIT，连接中断主动方的状态。

收到了对方的FIN报文，进入该状态，并发回最后一个ACK报文。然后等待一段时间（2倍的MSL），确保对方已收到该报文。

- CLOSE_WAIT，连接中断被动方的状态。

收到对方的FIN报文后进入该状态，随后发出其最后需要发送的数据。

- LAST_ACK，连接中断被动方的状态。

发送完最后需要发送的数据后，最后发送FIN报文，进入该状态，等待对方最后一个ACK报文。

- CLOSED，连接中断被动方的状态。

收到对方最后一条ACK后，进入该状态。


![网络数据分析](images\07\TCP_CLOSE.svg)

连接可以是“半开”，在这种情况下，一方已终止其结束，但另一方没有。已终止的一方不能再向连接发送任何数据，但另一方可以。终止方应继续读取数据，直到另一方终止。


### TCP重置

在理想情况中，每个连接都会以TCP终止来正常地结束。但在现实中，连接经常会突然断掉。

例如：攻击者正在进行端口扫描，或者主机配置错误等。这时接收端往往会使用RST标志的TCP数据包。

RST标志用来指出连接被异常中止或拒绝连接请求。



#### 本节实验操作

- 打开tcp_handshake.pcap样本文件
- 完成以下表格

|编号|源IP地址|源端口号|目的IP地址|目的端口号|标志位字段值|Sequence number|Ack number|当前握手阶段|
|-|-|-|-|-|-|-|-|-|
|1|
|2|
|3|

- 打开tcp_teardown.pcap样本文件
- 完成以下表格


|编号|源IP地址|源端口号|目的IP地址|目的端口号|标志位字段值|Sequence number|Ack number|当前撤销连接阶段|
|-|-|-|-|-|-|-|-|-|
|1|
|2|
|3|
|4|

- 打开tcp_refuseconnection.pcap样本文件
- 找到其中的TCP重置连接数据分组，完成下列表格：

|源IP地址|源端口号|目的IP地址|目的端口号|标志位字段值|Sequence number|Ack number|当前撤销连接阶段|
|-|-|-|-|-|-|-|-|
||

## UDP用户数据报协议

用户数据报协议（UDP）是Internet协议套件的核心成员之一。该协议由David P. Reed于1980年设计，并在RFC  768中正式定义。使用UDP，计算机应用程序可以将消息（在本例中称为数据报）发送到Internet协议（IP）网络上的其他主机。

### UDP与TCP对比

传输控制协议是面向连接的协议，这意味着它需要握手来建立端到端通信。一旦建立了连接，就可以通过连接双向发送用户数据。

- TCP
  - 可靠: 严格仅在传输层，TCP管理消息确认，重传和超时。进行了多次传递消息的尝试。如果它在途中丢失，服务器将重新请求丢失的部分。在TCP中，没有丢失数据，或者在多次超时的情况下，连接被丢弃。（然而，这种可靠性不包括应用层，在该应用层仍需要单独的确认流控制）
  
  - 已订购: 如果按顺序通过连接发送两条消息，则第一条消息将首先到达接收应用程序。当数据段以错误的顺序到达时，TCP缓冲区会延迟无序数据，直到所有数据都可以正确地重新排序并传送到应用程序。
  
  - 重量级: 在发送任何用户数据之前，TCP需要三个数据包来设置套接字连接。TCP处理可靠性和拥塞控制。
  
  - 流: 数据作为字节流读取，没有区别指示传输到信号消息（段）边界。

用户数据报协议是一种更简单的基于消息的无连接协议。无连接协议不会建立专用的端到端连接。通过在一个方向上从源到目的地发送信息而不验证接收器的准备状态或状态来实现通信。
- UDP
  - 不可靠：当发送UDP消息时，无法知道它是否会到达目的地; 它可能会在途中迷路。没有确认，重传或超时的概念。

  - 未订购：如果将两条消息发送给同一收件人，则无法预测它们到达的顺序。

  - 轻量级：没有消息排序，没有跟踪连接等。它是在IP之上设计的小型传输层。

  - 数据报： 数据包是单独发送的，只有在到达时才会检查其完整性。数据包具有明确的边界，这些边界在接收时受到尊重，这意味着接收器插槽上的读取操作将产生最初发送的整个消息。
  
  - 没有拥塞控制: UDP本身不会避免拥塞。拥塞控制措施必须在应用程序级别实施。
  
  - 广播：无连接，UDP可以广播 - 发送的数据包可以寻址到子网上的所有设备。

  - 多播：支持多播操作模式，从而可以自动路由单个数据报包，而无需复制到大量订户。

UDP以其简单、延迟小、数据传输效率高的优势，在越来越多场景下取代了TCP,如实时游戏、聊天程序qq、视频网站等。

- 网速的提升给UDP的稳定性提供可靠网络保障，丢包率很低，如果使用应用层重传，能够确保传输的可靠性。

- TCP为了实现网络通信的可靠性，使用了复杂的拥塞控制算法，建立了繁琐的握手过程，由于TCP内置的系统协议栈中，极难对其进行改进。

- 采用TCP，一旦发生丢包，TCP会将后续的包缓存起来，等前面的包重传并接收到后再继续发送，延时会越来越大，基于UDP对实时性要求较为严格的情况下，采用自定义重传机制，能够把丢包产生的延迟降到最低，尽量减少网络问题对游戏性造成影响。

### UDP头部

![UDP头部](images/07/udp头部.jpg)

UDP报头由4个域组成，其中每个域各占用2个字节，共8个字节。

- 源端口号
- 目标端口号
- 数据报长度
- 校验值


### 本节实验操作

- 打开upd_dnsrequest.pcap文件。
- 完成以下表格

|源IP地址|源端口号|目的IP地址|目的端口号|
|-|-|-|-|
||

- 按下图所示设置qq软件，记录服务器IP地址后，尝试抓取qq登录服务器的网络流量

![qq](images/07/qq网络设置1.png)
![qq](images/07/qq网络设置2.png)

- 使用显示过滤器ip.addr == 183.60.48.174（上述设置过程中确定的ip地址，可能因人而异），过滤分组
- 将捕获的数据包截图，填入实验报告中。